In [1]:
import pandas as pd
import numpy as np
import calendar
import os

from datetime import datetime
from elasticsearch import Elasticsearch,helpers
from elasticsearch.helpers import parallel_bulk
from elasticsearch.helpers import streaming_bulk
from es_pandas import es_pandas

# 导入datetime库
import datetime
import random

import modin.pandas as mpd
#import ray
#ray.shutdown()
#ray.init(num_cpus=4)

#import commond.ipynb from same folder
import import_ipynb
from common import *

importing Jupyter notebook from common.ipynb


In [2]:
import logging
import ray

logger = logging.getLogger(__name__)


ray.shutdown()
ray.init(num_cpus=4)


2020-07-05 12:49:16,667	INFO resource_spec.py:212 -- Starting Ray with 2.54 GiB memory available for workers and up to 1.27 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-07-05 12:49:16,973	INFO services.py:1148 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.200',
 'redis_address': '192.168.1.200:56932',
 'object_store_address': '/tmp/ray/session_2020-07-05_12-49-16_664218_480/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-07-05_12-49-16_664218_480/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-07-05_12-49-16_664218_480'}

In [5]:
@ray.remote
class Worker(object):
    def __init__(self):
        self.logger = logging.getLogger(__name__)
    def train(self):
        self.logger.warning("print from inside worker")
    def log_time(self, msg):
        # datetime object containing current date and time
        now = datetime.datetime.now()
        # dd/mm/YY H:M:S
        dt_string = now.strftime("%d/%m/%Y %H:%M:%S.%f")
        self.logger.info(msg)
        self.logger.info(dt_string)
    def get_interests_columns_local(self):
        cols = [
            'Weight',
            'Frequency',
            'Spreader_20FT',
            'Spreader_40FT',
            'Spreader_45FT',
            'Spreader_Twin',
            'OT_Control STS_Hoist Up (Controller)',
            'OT_Control STS_Hoist Down (Controller)',
            'OT_Control STS_Trolley Forward (Controller)',
            'OT_Control STS_Trolley Reverse (Controller)',
            'Phase A Current',
            'Phase B Current',
            'Phase C Current',
            'Average Current',
            'Neutral Current',
            'Voltage Vab',
            'Voltage Vbc',
            'Voltage Vca',
            '3 Phase Real Power',
            '3 Phase Reactive Power',
            '3 Phase Apparent Power',
            '3 Phase Positive Real Energy Used',
            '3 Phase Negative Real Energy Used',
            '3 Phase Positive Reactive Energy Used',
            '3 Phase Negative Reactive Energy Used',
            '3 Phase Apparent Energy',
            'OT_Control STS_Gantry Right (Controller)',
            'OT_Control STS_Gantry Left (Controller)',
            'OT_Control STS_Boom Up (Controller)',
            'OT_Control STS_Boom Down (Controller)',
            'Trolley Position',
            'Load Cell1 Data',
            'Load Cell2 Data',
            'Load Cell3 Data',
            'Load Cell4 Data',
            'SP STS_Landed']
        return cols        
    def ingest_bulk_dataframe_modin(self, start_date, end_date, crane_number):
        self.logger.info("start the ingest_bulk_dataframe_modin ")
        # 获取时间间隔
        delta = end_date - start_date

        ds = pd.DataFrame()
        const_ds = pd.DataFrame()

        const_ds.at[0, "CraneNumber"] = crane_number
        const_ds.at[1, "CraneNumber"] = crane_number
        const_ds.at[0, "Twistlock"] = 0
        const_ds.at[1, "Twistlock"] = 1
        const_ds.at[0, "Comm Status"] = 'OK'
        const_ds.at[1, "Comm Status"] = 'False'
        const_ds.at[0, "Asset Type"] = 'QC'
        const_ds.at[1, "Asset Type"] = 'YC'

        col_list = self.get_interests_columns_local.remote()

        return
        """
        # 遍历获得每一个时间点
        for i in range(int(delta.total_seconds())+1):
            #print_time("randint")
            ss = random.randint(10, 100)
            temp = start_date + datetime.timedelta(seconds=i)
            dt1 = temp + datetime.timedelta(milliseconds=ss)
            dt2 = dt1 + datetime.timedelta(milliseconds=500)

            #print_time("start create df ")
            df = pd.DataFrame(np.random.randint(10,1000,size=(2, 36)), columns=list(col_list))
            #print_time("finish create df ")

            df.at[0, "timestamp"] = dt1.strftime("%d/%m/%Y %H:%M:%S.%f")
            df.at[1, "timestamp"] = dt2.strftime("%d/%m/%Y %H:%M:%S.%f")

            #print_time("finish add df ")
            new_df = pd.concat([df, const_ds], axis=1)

            #print_time("finish cancat df ")
            ds = pd.concat([ds, new_df], axis=0)
            #print_time("finish append df ")
            del new_df
            #print_time("finish delete new_df ")
            del df

            if ds.shape[0] >= 14400:
               #print_time("start ingestion to ES ")
               log_time.remote("finish ingestion to ES ")
               msg = dt2.strftime("%d/%m/%Y %H:%M:%S.%f")
               print('Ingest %s  is completed!' % msg)
               ds = ds[0:0]

        if ds.shape[0] > 0:
           ds = ds[0:0] 
        return "done!"
        """
        





In [12]:
worker = Worker.remote()

crane = 'PQC' + str(random.randint(500, 900))
# 创建开始时间点和结束时间点
dt_start = datetime.datetime(2017,1,1,0,0,1)
dt_end =   datetime.datetime(2017,1,2,0,0,0)
worker.ingest_bulk_dataframe_modin.remote(dt_start, dt_end, crane)

ObjectID(8603ac10b8c093b315c675b2010000c801000000)

2020-07-05 12:52:58,911	ERROR worker.py:1012 -- Possible unhandled error from worker: ray::Worker.ingest_bulk_dataframe_modin() (pid=1115, ip=192.168.1.200)
  File "python/ray/_raylet.pyx", line 452, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 407, in ray._raylet.execute_task.function_executor
  File "<ipython-input-5-5d8a646a6673>", line 70, in ingest_bulk_dataframe_modin
AttributeError: 'function' object has no attribute 'remote'


In [4]:
# return dataframe by timerange
def ingest_bulk_dataframe(start_date, end_date, crane_number):
    # 获取时间间隔
    delta = end_date - start_date
    
    ds = pd.DataFrame()
    const_ds = pd.DataFrame()

    const_ds.at[0, "CraneNumber"] = crane_number
    const_ds.at[1, "CraneNumber"] = crane_number
    const_ds.at[0, "Twistlock"] = 0
    const_ds.at[1, "Twistlock"] = 1
    const_ds.at[0, "Comm Status"] = 'OK'
    const_ds.at[1, "Comm Status"] = 'False'
    const_ds.at[0, "Asset Type"] = 'QC'
    const_ds.at[1, "Asset Type"] = 'YC'
    
    col_list = get_interests_columns_simple()
    # 遍历获得每一个时间点
    for i in range(int(delta.total_seconds())+1):
        #print_time("randint")
        ss = 125 #random.randint(10, 100)
        temp = start_date + datetime.timedelta(seconds=i)
        dt1 = temp + datetime.timedelta(milliseconds=ss)
        dt2 = dt1 + datetime.timedelta(milliseconds=500)
        
        #print_time("start create df ")
        df = pd.DataFrame(np.random.randint(10,1000,size=(2, len(col_list))), columns=list(col_list))
        #print_time("finish create df ")
        
        df.at[0, "timestamp"] = dt1.strftime("%d/%m/%Y %H:%M:%S.%f")
        df.at[1, "timestamp"] = dt2.strftime("%d/%m/%Y %H:%M:%S.%f")

        #print_time("finish add df ")
        new_df = pd.concat([df, const_ds], axis=1)
        del df
        #print_time("finish cancat df ")
        ds = pd.concat([ds, new_df], axis=0)
        #ds = ds.append(new_df)
        #print(ds)
        #print_time("finish append df ")

        if ds.shape[0] >= 14400:
           #print_time("start ingestion to ES ")
           print_time("finish ingestion to ES ")
           msg = dt2.strftime("%d/%m/%Y %H:%M:%S.%f")
           print('Ingest %s  is completed!' % msg)
           ds = ds[0:0]
           
    if ds.shape[0] > 0:
       ds = ds[0:0] 


In [5]:
def print_time(msg):
    # datetime object containing current date and time
    now = datetime.datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S.%f")
    print(msg, dt_string)

In [9]:
crane = 'PQC' + str(random.randint(500, 900))
# 创建开始时间点和结束时间点
dt_start = datetime.datetime(2017,1,1,0,0,1)
dt_end =   datetime.datetime(2017,1,2,0,0,0)
ray.get(ingest_bulk_dataframe_modin.remote(dt_start, dt_end, crane))


In [6]:
@ray.remote
def f(x):
    return x * x

futures = [f.remote(i) for i in range(4)]
print(ray.get(futures))

[0, 1, 4, 9]


In [37]:


now = datetime.datetime.now()
# dd/mm/YY H:M:S
dt_string = now.astimezone().strftime("%Y-%m-%dT%H:%M:%S.%f%z")
print(dt_string)
print(now.astimezone().isoformat())

2020-07-05T20:11:10.645868+0800
2020-07-05T20:11:10.645868+08:00


In [ ]:
crane = 'PQC' + str(random.randint(500, 900))
# 创建开始时间点和结束时间点
dt_start = datetime.datetime(2017,1,1,0,0,1)
dt_end =   datetime.datetime(2017,1,2,0,0,0)
ingest_bulk_dataframe_modin(dt_start, dt_end, crane)
print('Ingest %s  is completed!' % crane)